In [2]:
!pip install tabulate

In [1]:
import datetime
import mysql.connector
from tabulate import tabulate
cnx = mysql.connector.connect(user='root', password='Likhitha@21', database='project')
cursor = cnx.cursor()

ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)

In [4]:
# 1. How many trainers are associated with each type of diet plan?
# And How many members are following each type of diet plan? And (keto, diabetic etc.)?

query1 = """
    SELECT W.DIET_PLAN, 
           COUNT(DISTINCT T.TRAINER_ID) AS NUM_OF_TRAINERS, 
           COUNT(DISTINCT M.MEMBER_ID) AS NUM_OF_MEMBERS 
    FROM TRAINER T 
    JOIN TRAINER_WORKOUT W ON T.TRAINER_ID = W.TRAINER_ID 
    LEFT JOIN MEMBERS M ON W.PLAN_ID = M.PLAN_ID 
    GROUP BY W.DIET_PLAN 
    ORDER BY NUM_OF_TRAINERS DESC, NUM_OF_MEMBERS DESC
"""

cursor.execute(query1)
result_set1 = cursor.fetchall()

headers1 = ["DIET_PLAN", "NUM_OF_TRAINERS", "NUM_OF_MEMBERS"]
table1 = tabulate(result_set1, headers=headers1)

print(table1)

DIET_PLAN               NUM_OF_TRAINERS    NUM_OF_MEMBERS
--------------------  -----------------  ----------------
Low Fat                               5                 8
High Protein                          4                 1
Diabetic                              3                 4
Keto                                  3                 4
High Protein Non Veg                  2                 3


In [5]:
#  2. What are the top peak hours?

# Prompts the user to enter the number of results they would want to view
limit = int(input("Enter the number of result you want to view for this query: "))

query2 = """
    SELECT EXTRACT(HOUR FROM CLOCK_TIME) AS HOUR_OF_DAY, COUNT(*) AS NUM_OF_VISITS
FROM (
  SELECT CLOCK_IN AS CLOCK_TIME
  FROM ATTENDANCE
  UNION ALL
  SELECT CLOCK_OUT AS CLOCK_TIME
  FROM ATTENDANCE
) AS ALL_VISITS
GROUP BY EXTRACT(HOUR FROM CLOCK_TIME)
ORDER BY NUM_OF_VISITS DESC
LIMIT {};
""".format(limit)

cursor.execute(query2)

result_set2 = cursor.fetchall()

headers2 = ["HOUR_OF_DAY", "NUM_OF_VISITS"]
table2 = tabulate(result_set2, headers=headers2)

print(table2)
#cnx.close()


Enter the number of result you want to view for this query: 1
  HOUR_OF_DAY    NUM_OF_VISITS
-------------  ---------------
           20               84


In [6]:
# 3. What are the 3 most commonly used payment methods from both expenses and payments?

# Prompts the user to enter the number of results they would want to view
limit = int(input("Enter the number of result you want to view for this query: "))

query3 = """
    SELECT PAYMENT_METHOD, SUM(NUM_TRANSACTIONS) AS TOTAL_TRANSACTIONS
FROM (
  SELECT PAYMENT_METHOD, COUNT(*) AS NUM_TRANSACTIONS
  FROM EXPENSES
  GROUP BY PAYMENT_METHOD
  UNION ALL
  SELECT PAYMENT_METHOD, COUNT(*) AS NUM_TRANSACTIONS
  FROM PAYMENTS
  GROUP BY PAYMENT_METHOD
) AS ALL_TRANSACTIONS
GROUP BY PAYMENT_METHOD
ORDER BY TOTAL_TRANSACTIONS DESC
LIMIT {};
""".format(limit)

cursor.execute(query3)

result_set3 = cursor.fetchall()

headers3 = ["PAYMENT_METHOD", "TOTAL_TRANSACTIONS"]
table3 = tabulate(result_set3, headers=headers3)

print(table3)

Enter the number of result you want to view for this query: 3
PAYMENT_METHOD      TOTAL_TRANSACTIONS
----------------  --------------------
cheque                              33
venmo                               29
credit_card                         27


In [7]:
# 4. How much of each equipment type does the gym currently have?

query4 = """
    SELECT G.EQUIPMENT_NAME, count(G.EQUIPMENT_NAME) as NUM_OF_EQUIPMENTS
FROM 
ORDERS O
INNER JOIN GYM_EQUIPMENTS G
ON O.GYM_EQUIPMENTS_ID = G.GYM_EQUIPMENTS_ID
WHERE O.STATUS = 'Delivered'
GROUP BY G.EQUIPMENT_NAME
ORDER BY NUM_OF_EQUIPMENTS DESC;
"""

cursor.execute(query4)

result_set4 = cursor.fetchall()

headers4 = ["EQUIPMENT_NAME", "NUM_OF_EQUIPMENTS"]
table4 = tabulate(result_set4, headers=headers4)

print(table4)

EQUIPMENT_NAME           NUM_OF_EQUIPMENTS
---------------------  -------------------
Training bench                           4
Dumbbell set                             4
Rowing machine                           3
Treadmill                                2
Stationary bicycle                       1
Barbell set                              1
Low impact treadmills                    1
Resistance bands                         1


In [8]:
# Fetch the count for a particular equipment asked by the user.
equipment_name = input("Enter the equipment name to see the count : ")
temp = {}

for name, count in result_set4:
    temp[name] = count
    
while(1):
    if equipment_name not in temp.keys():
        equipment_name = input('Enter correct equipment name')
    else:
        print(equipment_name, ':',temp[equipment_name])
        break

Enter the equipment name to see the count : Treadmill
Treadmill : 2


In [8]:
# 5 How many members is each trainer currently training?

query5 = """
SELECT T.TRAINER_FIRST_NAME, COUNT(M.MEMBER_ID) AS NUM_OF_MEMBERS
FROM TRAINER T
INNER JOIN TRAINER_WORKOUT W
ON T.TRAINER_ID = W.TRAINER_ID
INNER JOIN MEMBERS M
ON W.PLAN_ID = M.PLAN_ID
GROUP BY T.TRAINER_ID
ORDER BY NUM_OF_MEMBERS DESC;
"""

cursor.execute(query5)
result_set5 = cursor.fetchall()
headers5 = ["TRAINER_FIRST_NAME", "NUM_OF_MEMBER"]
table5 = tabulate(result_set5, headers = headers5)
print(table5)

TRAINER_FIRST_NAME      NUM_OF_MEMBER
--------------------  ---------------
Velvet                              7
Karry                               4
Shae                                4
Gracie                              2
Maud                                2
Frasquito                           1


In [9]:
# Fetch the count of memebers for perticular trainer.
trainer_name = input("Enter trainer name : ")
temp = {}

for name, count in result_set5:
    temp[name] = count
    
while(1):
    if trainer_name not in temp.keys():
        trainer_name = input('Enter correct trainer name')
    else:
        print(trainer_name, ':',temp[trainer_name])
        break

Enter trainer name : Velvet
Velvet : 7


In [9]:
# 6. How many members are signed up for each type of membership plan?

query6 = """
SELECT P.MEMBERSHIP_ID, P.DESCRIPTION, COUNT(M.MEMBER_ID) AS NUM_OF_MEMBERS
FROM MEMBERS M 
INNER JOIN MEMBERSHIP_PLAN P
ON M.MEMBERSHIP_ID = P.MEMBERSHIP_ID
GROUP BY M.MEMBERSHIP_ID
ORDER BY NUM_OF_MEMBERS DESC;
"""

cursor.execute(query6)
result_set6 = cursor.fetchall()
headers6 = ["MEMBERSHIP_ID", "DESCRIPTION", "NUM_OF_MEMBERS"]
table6 = tabulate(result_set6, headers = headers6)
print(table6)

  MEMBERSHIP_ID  DESCRIPTION                                          NUM_OF_MEMBERS
---------------  -------------------------------------------------  ----------------
              3  Half yearly membership plan with single payment                   4
              7  Yearly membership plan with half yearly payment                   4
              4  Half yearly membership plan with 3 months payment                 3
             10  Yearly membership plan with monthly payment                       3
              1  3 months membership plan with single payment                      2
              9  Yearly membership plan with 3 months payment                      2
              2  3 months membership plan with montly payment                      1
              5  Half yearly membership plan with monthly payment                  1


In [11]:
# Fetch the description and the number of members following that perticular membership plan
membership_id = int(input("Enter membership id (range of 1 to 10) : "))
while(1):
    if membership_id not in range(1,11):
        membership_id = int(input("Enter valid membership id (range of 1 to 10)"))
    else:
        break
flag = 0
for mid, desc, count in result_set6:
    if membership_id == mid:
        print('\nMembership plan id :',membership_id, '\nDescription is :', desc, '\nNumber of members following this membership plan :',count)
        flag = 1
if flag == 0:
    print('Membership plan with id', membership_id, 'does not have any members.')

Enter membership id (range of 1 to 10) : 1

Membership plan id : 1 
Description is : 3 months membership plan with single payment 
Number of members following this membership plan : 2


In [12]:
# 7. How does the amount of time each member spend in the gym compare to their scheduled hours for a week?

query7 = """
SELECT M.MEMBER_ID, MAX(W.SCHEDULE_HRS) AS SCHEDULE_HRS, SUM(HOUR(TIMEDIFF(A.CLOCK_OUT, A.CLOCK_IN))) AS TIME_SPEND
FROM ATTENDANCE A
INNER JOIN MEMBERS M
ON A.MEMBER_ID = M.MEMBER_ID
INNER JOIN TRAINER_WORKOUT W
ON M.PLAN_ID = W.PLAN_ID
WHERE DATE(A.CLOCK_IN) >= '2023-01-21' AND DATE(A.CLOCK_OUT) <= '2023-01-27'
GROUP BY M.MEMBER_ID
ORDER BY SCHEDULE_HRS DESC, TIME_SPEND DESC;
"""

cursor.execute(query7)
result_set7 = cursor.fetchall()
headers7 = ["MEMBER_ID", "SCHEDULE_HRS", "TIME_SPEND"]
table7 = tabulate(result_set7, headers = headers7)
print(table7)

  MEMBER_ID    SCHEDULE_HRS    TIME_SPEND
-----------  --------------  ------------
          3              20            12
         10              19            11
         15              18            20
          1              18            16
         12              18            12
         13              18            11
         11              18            10
          4              18             8
          7              16            14
         14              16            11
         20              16             8
         18              11             6
          9               9            13
          5               9            12
          8               4             5
         17               3            16
         16               3            12
         19               3            12
          6               3            11
          2               3             4


In [13]:
# Fetch the schedule hour & time spend on gym for a member.
member_id = int(input("Enter member id (range of 1 to 20) : "))
while(1):
    if member_id not in range(1,21):
        member_id = int(input("Enter valid membership id (range of 1 to 20)"))
    else:
        break
for mid, sched, time in result_set7:
    if member_id == mid:
        print('\nMember id :',member_id, '\nSchedule Hours are :',sched, '\nTime Spent in Gym :',time)

Enter member id (range of 1 to 20) : 20

Member id : 20 
Schedule Hours are : 16 
Time Spent in Gym : 8


In [14]:
# 8. Every rejected transaction and the associated member.

query8 = """
SELECT M.MEMBER_ID , M.MEMBER_FIRST_NAME, M.MEMBER_LAST_NAME, P.PAYMENT_ID, P.PAYMENT_AMOUNT, P.PAYMENT_DATE
FROM MEMBERS M
INNER JOIN PAYMENTS P
ON M.MEMBER_ID = P.MEMBER_ID
WHERE P.TRANSACTION_STATUS = 'fail'
ORDER BY P.PAYMENT_DATE;
"""

cursor.execute(query8)
result_set8 = cursor.fetchall()
headers8 = ["MEMBER_ID", "MEMBER_FIRST_NAME", "MEMBER_LAST_NAME", "PAYMENT_ID", "PAYMENT_AMOUNT", "PAYMENT_DATE"]
table8 = tabulate(result_set8, headers = headers8)
print(table8)

  MEMBER_ID  MEMBER_FIRST_NAME    MEMBER_LAST_NAME      PAYMENT_ID    PAYMENT_AMOUNT  PAYMENT_DATE
-----------  -------------------  ------------------  ------------  ----------------  --------------
         16  Mozes                Shattock                      47               265  2022-04-10
          6  Jere                 Tschursch                     48                60  2022-04-11
         19  Sim                  Langham                       49              1977  2022-04-12
         13  Tania                Tammadge                      50               346  2022-04-13
          6  Jere                 Tschursch                     51               332  2022-04-14
          7  Bryna                Cosley                        52               439  2022-04-15
         17  Alva                 Guerro                        53               872  2022-04-16
         14  Dinnie               Francesconi                   54              1077  2022-04-17
          1  Ameline    

In [15]:
# Fetch the member first name payment amount & payment date of rejected transaction for that perticular member id
member_id = int(input("Enter member id (range of 1 to 20) : "))
while(1):
    if member_id not in range(1,21):
        member_id = int(input("Enter valid member id (range of 1 to 20)"))
    else:
        break
flag = 0
count = 0
for mid, fname, lname, pid, pamt, date in result_set8:
    if member_id == mid:
        print('\nMember id :',member_id, '\nMember name is :', fname, '\nPayment amount :',pamt,'\nPayment date',date)
        flag = 1
        count = count + 1
if flag == 1:
    print('\nMember with id', member_id, 'has', count, 'rejected transactions.')
if flag == 0:
    print('Member with id', member_id, 'does not have any rejected transactions.')

Enter member id (range of 1 to 20) : 20

Member id : 20 
Member name is : Dolli 
Payment amount : 465.0 
Payment date 2022-07-29

Member id : 20 
Member name is : Dolli 
Payment amount : 768.0 
Payment date 2022-08-07

Member with id 20 has 2 rejected transactions.


In [16]:
# 9. Ratio of members to trainers

query9 = """
SELECT COUNT(DISTINCT M.MEMBER_ID) / COUNT(DISTINCT T.TRAINER_ID) AS MEMBER_TO_TRAINER_RATIO
FROM TRAINER T
JOIN TRAINER_WORKOUT W 
ON T.TRAINER_ID = W.TRAINER_ID
JOIN MEMBERS M
ON W.PLAN_ID = M.PLAN_ID;
"""

cursor.execute(query9)
result_set9 = cursor.fetchall()
headers9 = ["MEMBER_TO_TRAINER_RATIO"]
table9 = tabulate(result_set9, headers = headers9)
print(table9)


  MEMBER_TO_TRAINER_RATIO
-------------------------
                   3.3333


In [17]:
cursor.close()
cnx.close()